# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,opuwo,-18.0607,13.8400,20.02,12,0,1.60,NaN,1698729636
1,1,margaret river,-33.9500,115.0667,18.97,52,44,4.81,AU,1698729636
2,2,badger,64.8000,-147.5333,-0.86,93,75,0.00,US,1698729637
3,3,lihue,21.9789,-159.3672,25.77,75,40,3.60,US,1698729637
4,4,lichinga,-13.3128,35.2406,20.05,83,100,4.12,MZ,1698729637


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                         geo=True,
                                         tiles="OSM",
                                         frame_width=800,
                                         frame_height = 600,
                                         size = "Humidity",
                                         color="City"
                                       )

# Display the map

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values

choice_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
choice_city_df = choice_city_df.loc[(choice_city_df["Wind Speed"] < 4.5)]

# Drop any rows with null values

choice_city_df = choice_city_df.dropna()

# Display sample data
choice_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,lihue,21.9789,-159.3672,25.77,75,40,3.60,US,1698729637
27,27,adamstown,-25.0660,-130.1015,22.81,79,100,3.55,PN,1698729642
35,35,hawaiian paradise park,19.5933,-154.9731,24.73,94,100,1.54,US,1698729644
49,49,aitape,-3.1374,142.3475,25.71,88,100,1.75,PG,1698729647
50,50,cantagalo,-21.9811,-42.3681,21.48,99,100,0.56,BR,1698729648


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_copy_df = choice_city_df.copy()
hotel_df = hotel_copy_df[["City", "Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\prhoa\AppData\Local\Temp\ipykernel_35572\2685788591.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
3,lihue,US,21.9789,-159.3672,75,
27,adamstown,PN,-25.0660,-130.1015,79,
35,hawaiian paradise park,US,19.5933,-154.9731,94,
49,aitape,PG,-3.1374,142.3475,88,
50,cantagalo,BR,-21.9811,-42.3681,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "api_key": geoapify_key,
          "limit": 1
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
lihue - nearest hotel: Kauai Palms
adamstown - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
aitape - nearest hotel: No hotel found
cantagalo - nearest hotel: No hotel found
benjamin constant - nearest hotel: No hotel found
sal rei - nearest hotel: Hotel Marine Club
tabas - nearest hotel: هتل امیر
bardai - nearest hotel: No hotel found
shanwei - nearest hotel: 汕尾保利希尔顿逸林酒店
itaituba - nearest hotel: TG Hotel
iraucuba - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
pitimbu - nearest hotel: Pousada Brisa Mar
salinopolis - nearest hotel: Hotel Salinópolis
darnah - nearest hotel: فندق الفردوس
nova sintra - nearest hotel: Residência Ka Dencho
fada - nearest hotel: No hotel found
sakaraha - nearest hotel: Palace Club
isangel - nearest hotel: Tanna Lodge
aswan - nearest hotel: Yaseen hotel
perene - nearest hotel: No hotel found
porto novo -

,City,Country,Lat,Lng,Humidity,Hotel Name
3,lihue,US,21.9789,-159.3672,75,Kauai Palms
27,adamstown,PN,-25.0660,-130.1015,79,No hotel found
35,hawaiian paradise park,US,19.5933,-154.9731,94,No hotel found
49,aitape,PG,-3.1374,142.3475,88,No hotel found
50,cantagalo,BR,-21.9811,-42.3681,99,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng",
                                        "Lat",
                                         geo=True,
                                         tiles="OSM",
                                         frame_width=800,
                                         frame_height = 600,
                                         size = "Humidity",
                                         color="City",
                                         hover_cols=["Hotel Name", "Country"]
                                       )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)